In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import cv2
import os
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

In [0]:
def faceDetection(test_img):
    gray_img=cv2.cvtColor(test_img,cv2.COLOR_BGR2GRAY)
    faces_haar_cascade=cv2.CascadeClassifier('drive/My Drive/face_recognition/haarcascade_frontalface_default.xml')
    faces=faces_haar_cascade.detectMultiScale(gray_img,scaleFactor=1.3,minNeighbors=5)
    
    return faces,gray_img

In [0]:
def labels_for_training_data(directory):
    faces=[]
    faceID=[]
    
    for path,subdirnames,filenames in os.walk(directory):
        for filename in filenames:
            if filename.startswith("."):
                continue
            
            id = os.path.basename(path)
            img_path=os.path.join(path,filename)

            test_img=cv2.imread(img_path)
            if test_img is None:
                continue
            faces_rect,gray_img=faceDetection(test_img)
            if len(faces_rect)!=1:
                continue
            (x,y,w,h)=faces_rect[0]
            roi_gray=test_img[y:y+h,x:x+w]
            print("img path: ",img_path)
            print("ID",id)
            gray_img=cv2.cvtColor(roi_gray,cv2.COLOR_BGR2GRAY)
            faces.append(gray_img)
            faceID.append(int(id))
    
    return faces,faceID


In [0]:
def train_classifier(faces,faceID):
    face_recognizer=cv2.face.LBPHFaceRecognizer_create()
    face_recognizer.train(faces,np.array(faceID))
  #  face_recognizer.train(faces,faceID)
    return face_recognizer


In [0]:
def draw_rect(test_img,face):
    (x,y,w,h)=face
    cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=3)

In [0]:
faces,faceID=labels_for_training_data('drive/My Drive/face_recognition/photo')
face_recognizer=train_classifier(faces,faceID)
#face_reognizer.save('trainingData.yml')
name={0:'Kusum',1:'Priya'}

In [0]:
face_recognizer1=cv2.face.LBPHFaceRecognizer_create()
#face_recognizer1 = cv2.face.createLBPHFaceRecognizer()
test_img=cv2.imread('drive/My Drive/face_recognition/Kusum/7.jpg')
face_detected,gray_img=faceDetection(test_img)
plt.imshow(gray_img)
print("Detected face",face_detected)

In [0]:

for face in face_detected:
    (x,y,w,h)=face
    roi_img=gray_img[y:y+h,x:x+w]
    label,confidence=face_recognizer.predict(roi_img)
    print('confidence:',confidence)
    print('label:',label)
    draw_rect(test_img,face)
    predicted_name=name[label]
    cv2.putText(test_img,predicted_name,(x,y),cv2.FONT_HERSHEY_DUPLEX,2,(255,0,0),3)
     
    
resized_img=cv2.resize(test_img,(700,700))
plt.imshow(resized_img)
